# Vienna water areas

the row vector datasets :

FMZK – Wertebereich Attribut GEWAESSER


- 1 Donau
- 2 Donaukanal
- 3 Alte Donau
- 4 Neue Donau
- 5 Hafengewässer
- 6 Wienfluss
- 7 Liesing
- 8 Marchfeldkanal
- 9 Sonstiges Fließgewässer
- 10 Stehendes Gewässer 



were already mergen in QGIS and stored as a 10m raster file in 31256 projection:
"N:\C2205_FAIRiCUBE\f02_data\d050_data_WIEN\Mehrzweckkarte\FMZK_WIEN_GP\fmzk_gewaesser_poly_shp31256.shp"


## Option 1 - using UNION of vector datasets: 




In [11]:
## reading files....

# geometry:
import geopandas as gpd
import os
##input:
in_water_vector_datasest = r"N:\C2205_FAIRiCUBE\f02_data\d050_data_WIEN\Mehrzweckkarte\FMZK_WIEN_GP\fmzk_gewaesser_poly_shp31256.shp"
## reference poly: 100x100m:
reference_polygon = r"N:\C2205_FAIRiCUBE\f02_data\d050_data_WIEN\reference_units\GRID\grid31256\fishnet_100m_31256.shp"

## processing:
union_shp = os.path.splitext(in_water_vector_datasest)[0]+"_union.shp"
#print (union_shp)


## out:
out_water_100m_vector =         r"N:\C2205_FAIRiCUBE\f02_data\d050_data_WIEN\Vienna_gridded_data_collection\100m_GRID\Wien_water_areas_by_grid100m.shp"
out_water_100m_vector_indicator=r"N:\C2205_FAIRiCUBE\f02_data\d050_data_WIEN\Vienna_gridded_data_collection\100m_GRID\Wien_water_areas_by_grid100m_indicator.shp"


print ("datasets found")


# 1. transformation of land use vector dataset to the 100m Vienna GRID:
# 1 Load GRID
shp_100m_GRID = gpd.read_file(reference_polygon)
# 2 LOAD "land uise"
shp = gpd.read_file(in_water_vector_datasest)

# Perform spatial union
union_result = gpd.overlay(shp_100m_GRID, shp, how='union')
union_result.to_file(union_shp)
# Save the result to a new shapefile


print ("done")






datasets found
done


## Option 2 - using 10m raster as input


This approach is faster:
1. feature (poly) to 10m raster uisng majorty in QGI
2. Use the resulting 10m pixel for as input for OPTION 2 approach:

In [12]:
## location of 10m raster:

"N:\C2205_FAIRiCUBE\f02_data\d050_data_WIEN\Vienna_gridded_data_collection\10m_GRID\vienna_gewaesser_from_fmzk_31256_10m.tif"
"N:\C2205_FAIRiCUBE\f02_data\d050_data_WIEN\Vienna_gridded_data_collection\10m_GRID\vienna_gewaesser_mask1_from_fmzk_31256_10m.tif"


'N:\\C2205_FAIRiCUBE\x0c02_data\\d050_data_WIEN\\Vienna_gridded_data_collection\x08m_GRID\x0bienna_gewaesser_from_fmzk_31256_10m.tif'

In [18]:

## zonal statistics: output = vector dataset
import geopandas as gpd
from rasterstats import zonal_stats

input_10m_raster = r"N:\C2205_FAIRiCUBE\f02_data\d050_data_WIEN\Vienna_gridded_data_collection\10m_GRID\vienna_gewaesser_mask1_from_fmzk_31256_10m.tif"

zones  = reference_polygon
values = input_10m_raster
gdf = gpd.read_file(zones)
print(gdf.head())

stats = gpd.GeoDataFrame(zonal_stats(gdf, values, stats=["min", "max", "mean",'sum']))
gdf = gdf.join(stats)

print(gdf.head())



#gdf.to_file(out_water_100m_vector)
print ("done")

      EAST     NORTH                          GRID100mID  \
0 -11200.0  331000.0  EPSG31256_GRID_100m_E-11200N331000   
1 -11200.0  331100.0  EPSG31256_GRID_100m_E-11200N331100   
2 -11200.0  331200.0  EPSG31256_GRID_100m_E-11200N331200   
3 -11200.0  331300.0  EPSG31256_GRID_100m_E-11200N331300   
4 -11200.0  331400.0  EPSG31256_GRID_100m_E-11200N331400   

                                            geometry  
0  POLYGON ((-11200.000 331000.000, -11200.000 33...  
1  POLYGON ((-11200.000 331100.000, -11200.000 33...  
2  POLYGON ((-11200.000 331200.000, -11200.000 33...  
3  POLYGON ((-11200.000 331300.000, -11200.000 33...  
4  POLYGON ((-11200.000 331400.000, -11200.000 33...  
      EAST     NORTH                          GRID100mID  \
0 -11200.0  331000.0  EPSG31256_GRID_100m_E-11200N331000   
1 -11200.0  331100.0  EPSG31256_GRID_100m_E-11200N331100   
2 -11200.0  331200.0  EPSG31256_GRID_100m_E-11200N331200   
3 -11200.0  331300.0  EPSG31256_GRID_100m_E-11200N331300   
4 -11200.

# reclassify data frame (all water pixeld to valuea 1)

In [19]:
gdf.to_file(out_water_100m_vector)

# Export vector data to raster:

In [21]:
##

## raster to vector using vector attriubte (working perfectly):

import geopandas as gpd
import rasterio
from rasterio.features import rasterize
from rasterio.transform import from_bounds

# Load the polygon layer
polygons = gpd.read_file(out_water_100m_vector)

output_raster = r"N:\C2205_FAIRiCUBE\f02_data\d050_data_WIEN\Vienna_gridded_data_collection\100m_GRID\r07_Wien_water_areas_by_grid100m_indicator.tif"  


# Define the attribute to use for raster values
attribute_to_rasterize = 'sum'

# Define raster properties
#xmin, ymin, xmax, ymax = polygons.total_bounds

xmin=	-11200
ymin=	331000
xmax=	18800
ymax=	354000

projection ='epsg:31256'


pixel_size = 100  # Adjust as needed


width = int((xmax - xmin) / pixel_size)
height = int((ymax - ymin) / pixel_size)
transform = from_bounds(xmin, ymin, xmax, ymax, width, height)

# Create an empty array to hold rasterized values
raster_array = rasterize(
    [(geom, value) for geom, value in zip(polygons.geometry, polygons[attribute_to_rasterize])],
    out_shape=(height, width),
    transform=transform,
    fill=0,
    all_touched=False,  # or True
    dtype=rasterio.float32
)

# Write the raster to a GeoTIFF file
with rasterio.open(output_raster, "w", driver="GTiff", 
                   height=raster_array.shape[0], 
                   width=raster_array.shape[1], 
                   count=1, 
                   dtype=raster_array.dtype, 
                   crs=polygons.crs, 
                   transform=transform
                   ) as dst:
    dst.write(raster_array, 1)


print ("done")

done


## 